In [22]:
## extracting tag names and resource names from the dataset
import re
import pandas as pd
def extracting_tags(data, lang):
    dic = {}

    #data loading and spliting using regex
    with open(data,'rt') as myfile:
        language = ['ta','ka','ml','bn','te','hi','pa','gu']
        keyword = ['@'+lang]
        if lang not in language:
            return print('invalid language key')
        else:
            pass
        pattern = re.compile('|'.join(keyword))
        for line in myfile:
            if pattern.search(line):
                data = line.split('>')
                tags = data[1]
                resources = data[2]
                tag_str = str(tags) # to use 're' we need to pass the string 
                resource_str = str(resources)

                #extracting the tag name and resource name
                re_tag = 'dbpedia.org/property/([\w\W]+)'
                re_resource = '[^[a-zA-Z\\\'\"@#&,:;=.'']+'
                keys = re.findall(re_tag, tag_str)
                if keys:
                    key = keys[0]
                    values = re.findall(re_resource, resource_str)
                    value =  " ".join([value for value in values if len(value.strip()) > 0])

                    if len(value) > 1:
                        if key in dic:
                            dic[key].append(value)
                        else:
                            dic[key] = [value]
                            
        

    return dic

## defining data and language code 
data = 'data_bn'
lang_choice = "bn"
d = extracting_tags(data,lang_choice)

# converting list of string to single string 
new = {}
for key,value in d.items():
    new[key] = key
    value = '&&&'.join(value)
    new[key] = value
    
# creating pandas dataframe
df = pd.DataFrame.from_dict(new.items())
df.columns = ['tag','resource']

def search_words(word):
    result = re.findall(r'[A-Za-z0-9]+', word)
    return " ".join(result)

df['is_english']= df['tag'].apply(lambda x: 'yes' if (x == search_words(x)) else 'no') 

import torch
#use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0")

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
model.to(device)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

# translate Tamil to English

def translation(x,lang):
    tokenizer.src_lang =  lang +"_IN"
    encoded_ln = tokenizer(x, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded_ln)
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return(result)

#translation('பிப்ரவரி ')

newlist = []
for i,j in zip(df['is_english'],df['tag']):
    if (i == 'yes'):
        newlist.append(j)
    else:
        trans = translation(j,lang_choice)
        newlist.append(trans)
        
#newlist

df['translated_tag'] = newlist

#df[45:90]

df.to_csv('bengali.csv')

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [23]:
df[310:350]

,tag,resource,is_english,translated_tag
310,photoCaption,অগুং পাহাড় ১৯৮৯ সালে&&&অন্নপূর্ণা ১-এর দৃশ্য&...,yes,photoCaption
311,prominenceRef,(৮৭তম)&&&( 94 )&&&(১০০ তম)&&&( 2 )&&&(অবস্থা...,yes,prominenceRef
312,translation,সর্বশ্রেষ্ঠ বৃহৎ পর্বত&&&అంతర వలయ రహదారి గుం...,yes,translation
313,restingPlace,নোরা বেগ্রাফনিংস্প্লাৎসেন&&&টলোচেনাজ সিমেটারি ...,yes,restingPlace
314,প্রথমাংশ,মাইকেল&&&রেমন্ড&&&কেন&&&পিটার&&&জন&&&নরেন্দ্র ...,no,[The first one.]
315,শেষাংশ,উইলিয়ামস&&&মেয়ার&&&অকস ই&&&ক্রসওয়েল&&&পেনজ...,no,[The rest.]
316,সম্পাদকপ্রথমাংশ,টনি&&&জর্জ&&&কলিন&&&á é&&&ক্যাথরিন&&&আর এম ড...,no,[The president's share.]
317,সম্পাদকশেষাংশ,পিঙ্কনি&&&নিউন্স&&&লারকিন&&&মাসুদ&&&ডিক্সন&&&প...,no,[The last one.]
318,engine,উচ্চ শক্তির কঠিন জ্বালানীর একক পর্যায় সংমিশ্র...,yes,engine
319,filling,প্রচলিত উচ্চ বিস্ফোরক- একক অনুপ্রবেশ সাব-মনি...,yes,filling


In [ ]:
data = 'data.txt'
lang_choice = "ta"
d = extracting_tags(data,lang_choice)

In [ ]:
# converting list of string to single string 
new = {}
for key,value in d.items():
    new[key] = key
    value = '&&&'.join(value)
    new[key] = value
    

In [ ]:
# creating pandas dataframe
df = pd.DataFrame.from_dict(new.items())
df.columns = ['tag','resource']
#df

In [ ]:
def search_words(word):
    result = re.findall(r'[A-Za-z0-9]+', word)
    return " ".join(result)

In [ ]:
df['is_english']= df['tag'].apply(lambda x: 'yes' if (x == search_words(x)) else 'no') 

# Installation

1. transformers
2. tokenizer
3. sentencepiece (ater installing restart the kernel)

In [ ]:
import torch
#use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0")

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

In [ ]:
model.to(device)

In [ ]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

In [ ]:
# translate Tamil to English

def translation(x,lang):
    tokenizer.src_lang =  lang +"_IN"
    encoded_ln = tokenizer(x, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded_ln)
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return(result)

#translation('பிப்ரவரி ')

In [ ]:
newlist = []
for i,j in zip(df['is_english'],df['tag']):
    if (i == 'yes'):
        newlist.append(j)
    else:
        trans = translation(j,lang_choice)
        newlist.append(trans)
        

In [ ]:
#newlist

df['translated_tag'] = newlist

#df[45:90]

In [ ]:
df.to_csv('tamil.csv')